- Usaremos diversas bibliotecas: nltk, pandas e scikit-learn
- Lembre se que algumas bibliotecas devem ser instaladas!
- Usei "pip install *nome da biblioteca*" pra fazer a instalação

In [105]:
import nltk
import re
from nltk.tokenize import word_tokenize

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [ ]:
# Ler o arquivo com os dados, mostre uma amostra do arquivo e exibe a
# contagem de cada uma das colunas.

In [113]:
#dataset = pd.read_csv('/Users/rodrigomiani/Dropbox/Trabalho/UFU/Disciplinas/Organização e Recuperação da Informação/Exercícios/TP/TP4/Tweets_Mg.csv',encoding='utf-8')
dataset = pd.read_csv('reforma_previdencia_rotulado.csv', encoding="utf-8", delimiter=";")

In [108]:
dataset.head()

,Tweet,Classificação
0,Pressionem Queremos a reforma da previdência d...,Positivo
1,#EPTV1 Parabéns pela imparcialidade. Mostrando...,Positivo
2,@dep_paulinho TU É UM CANALHA. INDO CONTRA A R...,Positivo
3,"Em mais um golpe do Centrão, aquela grande fre...",Positivo
4,Eles acham que o objetivo é só lula livre e ca...,Positivo


In [114]:
dataset.count()

Tweet            2232
Classificação    2232
dtype: int64

In [ ]:
# Vamos contar quantos tweets de cada tipo existem: neutro, positivo e negativo.

In [115]:
dataset[dataset.Classificação=='Neutro'].count()

Tweet            780
Classificação    780
dtype: int64

In [116]:
dataset[dataset.Classificação=='Positivo'].count()

Tweet            740
Classificação    740
dtype: int64

In [117]:
dataset[dataset.Classificação=='Negativo'].count()

Tweet            712
Classificação    712
dtype: int64

In [ ]:
# Precisamos criar váriaveis diferentes para armazenar os tweets e a sua classificação

In [120]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

tweets_normal = dataset['Tweet'].values
classes = dataset['Classificação'].values


tweets = []
for tweet in tweets_normal:
    palavras = tweet.lower().split()
    filtrar_palavras= [w for w in palavras if w not in stopwords]
    tweet_pronto = ' '.join(filtrar_palavras)
    tweets.append(tweet_pronto)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Vamos treinar o nosso primeiro modelo de classificação de texto.
# Algumas coisas são importantes aqui:

# 1) Precisamos definir como representar os tweets (BoW)
# 2) Precisamos definir qual algoritmo de classificação será usado


### Breve explicação sobre o funcionamento de algoritmos de classificação: ###
# O algoritmo irá relacionar o conteúdo da BoW (1,0,2,0,1) com a respectiva classificação 'Neutro'. Usando diferentes critérios e com auxílio dos dados
# analisados, o algoritmo irá criar "regras" para identificar/generalizar cada uma das classificações - Neutro, Positivo e Negativo. Chamamos esse conjunto de regras de "modelo".
# Feito isso, quando o modelo receber um novo tweet (BoW) sem a classificação, com base nas regras que foram criadas, ele irá tentar "adivinhar" qual será a classe daquele tweet.
# ####

In [ ]:
# Para resolver o problema 1), vamos usar somente o TF - CountVectorizer. Essa função já limpa aqueles caracteres esquisitos que vimos lá em cima.

# Na linha 1, criamos um objeto do tipo CountVectorizer chamado vectorizer. Após isso, na linha 2, usamos o objeto vectorizer para calcular a frequência de todas as palavras da lista de tweets e armazenamos seu retorno em freq_tweets.

In [121]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)

In [ ]:
# Para resolver o problema 2), vamos trabalhar com um algoritmo de classificação chamado de Naive Bayes. Ele é baseado em probabilidades.
# Na linha 1, criamos um objeto chamado modelo do tipo Naive Bayes Multinomial.
# Na linha 2, treinamos o modelo usando a frequência de palavras (freq_tweets) e as classes de cada instância.

In [122]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB()

In [ ]:
# Vamos fazer alguns testes "manuais". Ou seja, fornecer como entrada para o modelo alguns tweets
# e deixar que ele faça a classificação. Na opinião de vocês, qual seria a classificação para cada um desses
# tweets?

In [123]:
testes = ['O governo de Minas é uma tragédia, muito ruim','Estou muito feliz com o governo de Minas esse ano','O estado de Minas Gerais decretou calamidade financeira!!!','A segurança do estado está deixando a desejar','O governador de Minas é do Novo']
print(testes)

['O governo de Minas é uma tragédia, muito ruim', 'Estou muito feliz com o governo de Minas esse ano', 'O estado de Minas Gerais decretou calamidade financeira!!!', 'A segurança do estado está deixando a desejar', 'O governador de Minas é do Novo']


In [ ]:
# Calculo a BoW dos tweets dentro da variável testes usando o TF.

In [124]:
freq_testes = vectorizer.transform(testes)

In [ ]:
# Faço a classificação dos tweets de testes usando o modelo treinado.

In [125]:
modelo.predict(freq_testes)

array(['Negativo', 'Neutro', 'Negativo', 'Positivo', 'Neutro'],
      dtype='<U8')

In [ ]:
# Vimos que o modelo funciona! Tem alguns erros mas isso faz parte do processo.
# O próximo passo agora é fazer uma avaliação mais robusta do modelo.
# Vamos usar uma parte da base de dados para treinar e a outra parte para testar.
# Uma maneira de se fazer isso é usando um método chamado de "Cross Validation" ou "Validação cruzada".
# Essta técnica consiste em dividir todo o conjunto de dados em K partes, que serão chamadas de folds.
# Dessas partes, uma será separada para teste e as outras restantes serão usadas para treinar o modelo.

## Exemplo ##

# Para k = 10 , imagine que todo nosso dado de treino foi dividido em 10 partes distintas.
# Assim, o modelo será treinado com 9 partes, e testado com a parte restante. Esse processo é repetido até que o modelo seja treinado e testado com todas as partes do dado.

# A variável "resultados" guarda as previsões feitas pelo pelo modelo usando a validação cruzada.

In [126]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [ ]:
# Pronto! Modelo treinado e validado! Como descobrir o desempenho do modelo? Inicialmente, usaremos uma
# medida chamada de Acurácia que nada mais é do que o percentual de acertos que o modelo teve.

In [127]:
metrics.accuracy_score(classes,resultados)

0.6084229390681004

In [ ]:
# E se eu quiser saber o desempenho por cada uma das classes? Talvez o modelo acerte mais uma classe
# do que a outra...

In [128]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.59      0.56      0.58       712
      Neutro       0.68      0.54      0.60       780
    Positivo       0.57      0.73      0.64       740

    accuracy                           0.61      2232
   macro avg       0.61      0.61      0.61      2232
weighted avg       0.62      0.61      0.61      2232



In [ ]:
# E seu eu quiser saber a quantidade de acertos por classe? Nesse caso precisamos mostrar
# a matriz de confusão.

In [129]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo       401     121       190   712
Neutro         151     420       209   780
Positivo       122      81       537   740
All            674     622       936  2232


In [ ]:
# E aí? Terminou? Satisfeito com os resultados? Será que não dá pra melhorar?
# Um primeiro ponto seria alterar a BoW. Uma outra forma de modelar isso é usando o conceito de n-grams.
## Exemplo ##
# Na frase: “Eu não gosto desse governo”, na modelagem inicial, passamos para o modelo cada palavra sendo uma feature, ficaria
# a) {eu, não, gosto, desse, governo}
# Usando Bigrams, passaríamos para o modelo 2 palavras, veja:
# b) {eu não, não gosto, gosto desse, desse governo}

## Ou seja, estamos dizendo que uma palavra tem alguma relação com outra palavra que vem logo a seguir. Lembra a história da independência de termos?

## Para implementar o bigrama basta usar o parâmetro ngram_range a seguir.
## A documentação da função CountVectorizer diz o seguinte:
# For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

In [130]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [131]:
metrics.accuracy_score(classes,resultados)

0.6111111111111112

In [132]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.60      0.57      0.58       712
      Neutro       0.66      0.56      0.61       780
    Positivo       0.58      0.70      0.64       740

    accuracy                           0.61      2232
   macro avg       0.61      0.61      0.61      2232
weighted avg       0.62      0.61      0.61      2232



In [133]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo       405     126       181   712
Neutro         146     439       195   780
Positivo       124      96       520   740
All            675     661       896  2232
